# Main file for thesis project

Ie the top level script which runs the whole `pic -> recognise -> aim gimbal -> repeat` show

In [ ]:
#!/usr/bin/python3
# top level script to run the show
import time
import PIL.Image

import image_classifier
import gimbal_control as gc

import matplotlib.pyplot as plt

## A Basic Loop

In [ ]:
IC = image_classifier.ImageClassifier(
    graph_file='../Models/MobileNet_SSD_caffe/graph',
    label_file='../Models/MobileNet_SSD_caffe/categories.txt',
    class_of_interest='person',
    camera_resolution=(320, 304))

In [ ]:
t_start = time.time()

yaw_before = []
yaw_after = []
angles_just_before = []
time_vals = []

while True:
    t = time.time()
    angles_before = gc.get_motor_angles()  # get the current motor angles - need these to calculate offsets
    img_array = IC.take_picture_and_start_inference()
#     time.sleep(0.02)
    angles_after = gc.get_motor_angles()
    output_dict, bb, bb_angles, inference_time_ms = IC.get_inference_result()
    
    if angles_before == 0: print('before'); continue
    if angles_after == 0: print('after'); continue
    
    if bb_angles == -1:
        print('-', end='')
        img = PIL.Image.fromarray(img_array)
        continue
        
    else:
        print('+', end='')
#         time.sleep(0.02)
        angle_yaw = angles_before['yaw']*0.5 + angles_after['yaw']*0.5    

        # draw a box on the object
        (x1, y1), (x2, y2) = bb
        img = PIL.Image.fromarray(img_array)
        img = PIL.Image.fromarray(image_classifier.draw_bounding_box(y1, x1, y2, x2, img_array))
#         plt.imshow(img)
#         plt.show()

        # convert angles of object to centroids
        (phi_x1, phi_y1), (phi_x2, phi_y2) = bb_angles
        phi_yaw = (phi_x1 + phi_x2)/2
        phi_pitch = (phi_y1 + phi_y2)/2
        
#         angles['pitch'] = -angles['pitch']
#         print('gimbal = %d, obj = %d' % (angles['pitch'], phi_pitch))

        angles2 = gc.get_motor_angles()
        if angles2 == 0: print('angles2'); continue
        
#         print('gc_pic = %d, gc_now = %d, phi_yaw = %d, target = %d' % (angles['yaw'], angles2['yaw'], phi_yaw, phi_yaw*0.5 + angles['yaw']))
#         print('phi_yaw = %d' % phi_yaw)
        gc.send_angle_command(0,
                              0,
                              phi_yaw*0.5 + angle_yaw)
    
    yaw_before.append(angles_before['yaw'])
    yaw_after.append(angles_after['yaw'])
    angles_just_before.append(angles2['yaw'])    
    t_loop = time.time() - t
    time_vals.append(time.time() - t_start)
    
    if t_loop < 0.25:
        time.sleep(0.25 - t_loop)
    else:
        print(t_loop)
        # old integrator approach
#         x_target += (x1+x2)/2
#         x_target = max(-20, min(20, x_target))
#         gc.send_angle_command(0, 0, x_target)
#     time.sleep(0.05)

In [ ]:
t_start = time_vals[0]
time_vals = [t - t_start for t in time_vals]
plt.plot(time_vals, yaw_before, label='yaw_before')
plt.plot(time_vals, yaw_after, label='yaw_after')
plt.plot(time_vals, angles_just_before, label='angles_just_before')
plt.legend()
plt.grid()
fig = plt.gcf(); fig.set_size_inches(18.5, 5, forward=True)
plt.show()

In [ ]:
plt.plot(time_vals, [i - j for i, j in zip(yaw_before, yaw_after)],
         label='difference between samples before and after')
fig = plt.gcf(); fig.set_size_inches(18.5, 5, forward=True); plt.grid(); plt.legend(); plt.show()

In [ ]:
avg_angles = [(i+j)/2 for i, j in zip(yaw_before, yaw_after)]
avg_angles_velocity = [(j-i)/(t_a-t_b) for i,j,t_b,t_a in zip(avg_angles[0:-1], avg_angles[1:], time_vals[0:-1], time_vals[1:])]

# avg_angles_velocity = [(i+j)/2 for i,j in zip(avg_angles_velocity[0:-1], avg_angles_velocity[1:])]
# avg_angles_velocity.append(avg_angles_velocity[-1])
plt.plot(time_vals[1:], avg_angles_velocity, label='est angular velocity')
plt.plot(time_vals, yaw_before, label='yaw_before')
fig = plt.gcf(); fig.set_size_inches(18.5, 5, forward=True); plt.grid(); plt.legend(); plt.show()

In [ ]:
img_array = IC.take_picture_and_start_inference()
output_dict, bb, bb_angles, inference_time_ms = IC.get_inference_result(show_image=True)

if bb_angles == -1:
    print('-', end='')
    img = PIL.Image.fromarray(img_array)

else:
    print('+', end='')
    # draw a box on the object
    (x1, y1), (x2, y2) = bb
    img = PIL.Image.fromarray(img_array)
    img = PIL.Image.fromarray(image_classifier.draw_bounding_box(y1, x1, y2, x2, img_array))

img

In [ ]:
IC.__del__()
del IC